# Homework 1

Solve problem 2 parts b through d.

## Part 2b

Write a simulator that can randomly generate sequences of "weathers" from the state transition function. 

In [2]:
import numpy as np
import random

# Transition matrix: 
# Rows represent today's weather in the sequence of sunny/cloudy/rainy
# Columns represent tomorrow's weather in the sequence of sunny/cloudy/rainy
transitionMatrix = np.array([[0.8, 0.2, 0], 
                            [0.4, 0.4, 0.2], 
                            [0.2, 0.6, 0.2]])
# List of States
states = ['sunny','cloudy','rainy']

In [27]:
def weather_simulator(iterations):
    # Initialize at random state
    currState = random.choice(states)
    weather = [currState]

    for i in range(iterations-1):
        if currState == 'sunny':
            nextStateProb = transitionMatrix[0]
        elif currState == 'cloudy':
            nextStateProb = transitionMatrix[1]
        elif currState == 'rainy':
            nextStateProb = transitionMatrix[2]

        # Get a random next state based on the current state
        nextState = random.choices(states, nextStateProb)
        weather.append(nextState[0])
        
        currState = nextState[0]
        
    return weather

In [35]:
# Simulate 20 days of weather based on the transition matrix
weatherSequence = weather_simulator(20)
print(weatherSequence)

['cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'rainy', 'cloudy', 'cloudy', 'rainy', 'cloudy', 'rainy', 'sunny']


## Part 2c
Use your simulator to determine the stationary distribution of this Markov chain. The stationary distribution measures the probability that a random day will be sunny, cloudy, or rainy.

(i)     How to initialize the simulation

(ii)    How many transitions should be considered before taking a result

(iii)   How many simulation runs were conducted to arrive at the statistics for the stationary distribution

## Part 2d
Can you devise a closed-form solution to calculate the stationary distribution based on the state transition matrix above?

Perform the diagonalization of the state matrix A to get the same result